In [1]:
import pandas as pd
import numpy as np

from src.language_processing2 import processing_text
from src.language_processing2 import vectorize_text
from src.language_processing2 import vectorize_desc
from src.modeling import test_model
from src.modeling import test_model2
from src.modeling import test_model3

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split,KFold

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,1000)

In [4]:
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,1000)

In [5]:
df.head()

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,desc-TF-IDF-990,desc-TF-IDF-991,desc-TF-IDF-992,desc-TF-IDF-993,desc-TF-IDF-994,desc-TF-IDF-995,desc-TF-IDF-996,desc-TF-IDF-997,desc-TF-IDF-998,desc-TF-IDF-999
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
test_model3(df,'spam_hijack',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9201875000000002

Log-Loss score of this model is

0.25771578635491316

AUC score of this model is

0.8878695993945074



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9156244843571472

Log-Loss score of this model is

0.23974509975605765

AUC score of this model is

0.8888592257717483


In [20]:
df2 = df.sample(frac=1)
df2_training = df2[:16000]
df2_testing = df2[16000:]
y = df2_training['spam_marketing']
X = df2_training[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count','favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
model = GradientBoostingClassifier()
kf = KFold(n_splits=5, shuffle=True)

ll_performance = []
auc_performance = []
acc_performance = []
#kfold split on X_ (which is X_train of len 110)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    y_pred = model.predict_proba(X_test)
    log_ll = log_loss(y_test, y_pred)
    ll_performance.append(log_ll)
    auc = roc_auc_score(y_test,y_predict)
    auc_performance.append(auc)
    acc = accuracy_score(y_test,y_predict)
    acc_performance.append(acc)
        
        
print("\n" + "Score summary for initial test (first 80% of data)" + "\n")
print("\n" + "Accuracy score of this model is" + "\n")
print(np.mean(acc_performance))
print("\n" + "Log-Loss score of this model is" + "\n")
print(np.mean(ll_performance))
print("\n" + "AUC score of this model is" + "\n")
print(np.mean(auc_performance))
print("\n")

y = df2_testing['spam_marketing']
X = df2_testing[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count','favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    #model = model()
kf = KFold(n_splits=5, shuffle=True)

ll_performance = []
auc_performance = []
acc_performance = []
#kfold split on X_ (which is X_train of len 110)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    y_pred = model.predict_proba(X_test)
    log_ll = log_loss(y_test, y_pred)
    ll_performance.append(log_ll)
    auc = roc_auc_score(y_test,y_predict)
    auc_performance.append(auc)
    acc = accuracy_score(y_test,y_predict)
    acc_performance.append(acc)
        
        
print("\n" + "Score summary for final test (last 20% of data)" + "\n")
print("\n" + "Accuracy score of this model is" + "\n")
print(np.mean(acc_performance))
print("\n" + "Log-Loss score of this model is" + "\n")
print(np.mean(ll_performance))
print("\n" + "AUC score of this model is" + "\n")
print(np.mean(auc_performance))


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9459375

Log-Loss score of this model is

0.15139598728956508

AUC score of this model is

0.5664695799489499



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9440817432075242

Log-Loss score of this model is

0.16111093193419151

AUC score of this model is

0.5702437688831807


In [2]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,1000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,1000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9557500000000001

Log-Loss score of this model is

0.3110620137609102

AUC score of this model is

0.6946650794503848



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9488276811093366

Log-Loss score of this model is

0.5279932671318143

AUC score of this model is

0.6054930489899447


In [2]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,2000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,1000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.95525

Log-Loss score of this model is

0.3457592146184882

AUC score of this model is

0.6812271895237958



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9430826803154411

Log-Loss score of this model is

0.5746423567463161

AUC score of this model is

0.5844626490015536


In [3]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,1000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,2000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9561249999999999

Log-Loss score of this model is

0.32363300761625824

AUC score of this model is

0.6860286282080982



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9518223791333152

Log-Loss score of this model is

0.44047126689288546

AUC score of this model is

0.6208804094702127


In [4]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,2000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,2000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9551875000000001

Log-Loss score of this model is

0.3375067860078896

AUC score of this model is

0.6779583713407706



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9458308037646208

Log-Loss score of this model is

0.4871428145158421

AUC score of this model is

0.6151665385655251


In [5]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,3000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,2000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9530000000000001

Log-Loss score of this model is

0.30681134844151475

AUC score of this model is

0.6744085821459241



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9520717556919187

Log-Loss score of this model is

0.47260118772187615

AUC score of this model is

0.6079097903150857


In [6]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,2000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,3000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.95625

Log-Loss score of this model is

0.3547430040901224

AUC score of this model is

0.6814272143735747



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9400889162860638

Log-Loss score of this model is

0.5729450154787487

AUC score of this model is

0.5856622137338686


In [7]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,3000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,3000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.95425

Log-Loss score of this model is

0.3356319249364111

AUC score of this model is

0.6816079874208449



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9558158287178434

Log-Loss score of this model is

0.42671267787482653

AUC score of this model is

0.613681336357099


In [8]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,4000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,3000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.95425

Log-Loss score of this model is

0.3723684980506903

AUC score of this model is

0.6823367532701811



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9493258115634757

Log-Loss score of this model is

0.41695159330497306

AUC score of this model is

0.6130141243845816


In [9]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,3000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,4000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9555

Log-Loss score of this model is

0.390611276942315

AUC score of this model is

0.6882559037155062



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9493261228950096

Log-Loss score of this model is

0.5659464784085624

AUC score of this model is

0.5862045284921659


In [10]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,4000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,4000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9561874999999999

Log-Loss score of this model is

0.4094692064883764

AUC score of this model is

0.6895559070065984



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9450817400942089

Log-Loss score of this model is

0.40271715611834064

AUC score of this model is

0.5716808836322892


In [11]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,5000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,4000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.95525

Log-Loss score of this model is

0.3617570554819996

AUC score of this model is

0.6904283986430129



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9530683279317312

Log-Loss score of this model is

0.526356486566871

AUC score of this model is

0.6120179730193397


In [12]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,4000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,5000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.954375

Log-Loss score of this model is

0.3805663048004722

AUC score of this model is

0.6771277948451278



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9490751896787369

Log-Loss score of this model is

0.44971577121642337

AUC score of this model is

0.5888128086810697


In [13]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,5000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,5000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9560625

Log-Loss score of this model is

0.3600388187497185

AUC score of this model is

0.6813134728716808



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9460817369808936

Log-Loss score of this model is

0.4828053047509292

AUC score of this model is

0.6122243693853435


In [14]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,6000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,5000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9561875000000001

Log-Loss score of this model is

0.4226215491933868

AUC score of this model is

0.6902917501319835



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9428323697622363

Log-Loss score of this model is

0.4372546038778478

AUC score of this model is

0.5804095904970232


In [15]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,5000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,6000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.953375

Log-Loss score of this model is

0.34286716759827307

AUC score of this model is

0.6764620087110313



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9498251873437503

Log-Loss score of this model is

0.4205551289929936

AUC score of this model is

0.6013851527545973


In [16]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,6000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,6000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.955125

Log-Loss score of this model is

0.38607410645521445

AUC score of this model is

0.6794813177446667



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9453301826582109

Log-Loss score of this model is

0.5452616782224806

AUC score of this model is

0.5841566016411359


In [17]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,7000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,6000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9566250000000001

Log-Loss score of this model is

0.3811618753946305

AUC score of this model is

0.6851560641234984



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9518223791333152

Log-Loss score of this model is

0.47584810790394894

AUC score of this model is

0.6172322290728631


In [18]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,6000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,7000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9561875000000001

Log-Loss score of this model is

0.37989896498544046

AUC score of this model is

0.6867563152891556



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9450851647410812

Log-Loss score of this model is

0.4546017776825589

AUC score of this model is

0.596785641311778


In [19]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,7000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,7000)
test_model3(df,'spam_marketing',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9573750000000001

Log-Loss score of this model is

0.396181236891592

AUC score of this model is

0.6983796363762993



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9453367206204215

Log-Loss score of this model is

0.5624086981030103

AUC score of this model is

0.5605088895189769
